In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn import model_selection
from sklearn.svm import SVC

c:\Users\jahon\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data = pd.read_csv('dados.csv', sep=',')
display(data)

,MADE,SHOT TYPE,PERIOD,TIME REMAINING,SHOT DISTANCE (FT),is_home,off_win_rate,def_blocks,def_win_rate,Age,Height,Weight,DRAFT NUMBER
0,1,2,1,675,13,1,0.621951,435,0.621951,29,195.072,100.000000,6
1,0,3,1,402,22,1,0.621951,435,0.621951,29,195.072,100.000000,6
2,0,2,1,205,6,1,0.621951,435,0.621951,29,195.072,100.000000,6
3,1,2,2,453,9,1,0.621951,435,0.621951,29,195.072,100.000000,6
4,0,2,2,393,6,1,0.621951,435,0.621951,29,195.072,100.000000,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
209621,1,2,1,189,13,0,0.439024,460,0.560976,29,201.168,90.909091,40
209622,0,3,2,614,24,0,0.439024,460,0.560976,29,201.168,90.909091,40
209623,0,2,2,225,7,0,0.439024,460,0.560976,29,201.168,90.909091,40
209624,0,2,4,640,8,0,0.439024,460,0.560976,29,201.168,90.909091,40


### Normalizando os dados

In [4]:
data_normalizado = data.copy()
for column in data_normalizado.columns:
    data_normalizado[column] = (data_normalizado[column] - data_normalizado[column].min()) / (data_normalizado[column].max() - data_normalizado[column].min())

display(data_normalizado)

,MADE,SHOT TYPE,PERIOD,TIME REMAINING,SHOT DISTANCE (FT),is_home,off_win_rate,def_blocks,def_win_rate,Age,Height,Weight,DRAFT NUMBER
0,1.0,0.0,0.0,0.937500,0.158537,1.0,0.704545,0.634328,0.704545,0.416667,0.565217,0.461538,0.050505
1,0.0,1.0,0.0,0.558333,0.268293,1.0,0.704545,0.634328,0.704545,0.416667,0.565217,0.461538,0.050505
2,0.0,0.0,0.0,0.284722,0.073171,1.0,0.704545,0.634328,0.704545,0.416667,0.565217,0.461538,0.050505
3,1.0,0.0,0.2,0.629167,0.109756,1.0,0.704545,0.634328,0.704545,0.416667,0.565217,0.461538,0.050505
4,0.0,0.0,0.2,0.545833,0.073171,1.0,0.704545,0.634328,0.704545,0.416667,0.565217,0.461538,0.050505
...,...,...,...,...,...,...,...,...,...,...,...,...,...
209621,1.0,0.0,0.0,0.262500,0.158537,0.0,0.363636,0.727612,0.590909,0.416667,0.652174,0.307692,0.393939
209622,0.0,1.0,0.2,0.852778,0.292683,0.0,0.363636,0.727612,0.590909,0.416667,0.652174,0.307692,0.393939
209623,0.0,0.0,0.2,0.312500,0.085366,0.0,0.363636,0.727612,0.590909,0.416667,0.652174,0.307692,0.393939
209624,0.0,0.0,0.6,0.888889,0.097561,0.0,0.363636,0.727612,0.590909,0.416667,0.652174,0.307692,0.393939


### Dividindo X e Y para o modelo

In [28]:
X = np.array(data_normalizado.drop(['MADE'], axis=1))
y = np.array(data_normalizado['MADE'])

### Dividindo dados de treino e teste para o modelo

Usaremos cerca de 5% dos dados já que o modelo SVM não precisa de tantos dados

In [29]:
## selecionar mil pontos do array X de forma aleatória

indices = np.random.randint(0, len(X), 10000)

X = X[indices]
y = y[indices]

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, random_state=23, test_size=0.2)



In [36]:
X[0]

array([1.        , 0.2       , 0.54722222, 0.29268293, 0.        ,
       0.63636364, 0.30970149, 0.52272727, 0.08333333, 0.73913043,
       0.5       , 0.03030303])

In [30]:
indices

array([ 35331, 147857, 185777, ..., 190853, 189248,  35570])

In [40]:
# X = X[:1000]
# y = y[:1000]
# X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, random_state=23, test_size=0.2)


## SVM

In [31]:
param_grid = {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'kernel': ['rbf']}
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=3)
grid.fit(X_train, y_train)

grid_predictions = grid.predict(X_test)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.569 total time=   3.1s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.595 total time=   2.8s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.577 total time=   2.8s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.574 total time=   2.9s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.607 total time=   2.9s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.564 total time=   2.9s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.588 total time=   2.8s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.575 total time=   2.7s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.578 total time=   2.9s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.599 total time=   2.8s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.522 total time=   3.1s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

In [32]:
print('Ein: %0.4f' % (1 - accuracy_score(y_train, grid.predict(X_train))))
print('Eout: %0.4f' % (1 - accuracy_score(y_test, grid.predict(X_test))))
y_pred = grid.predict(X_test)

print(classification_report(y_test, y_pred))

Ein: 0.3640
Eout: 0.3765
              precision    recall  f1-score   support

         0.0       0.63      0.71      0.67      1078
         1.0       0.61      0.52      0.56       922

    accuracy                           0.62      2000
   macro avg       0.62      0.62      0.62      2000
weighted avg       0.62      0.62      0.62      2000



### Abaixo temos, com base nos testes do gridsearch, os melhores parâmetros C e gamma para o modelo

In [34]:
grid.best_params_

{'C': 1000, 'gamma': 0.1, 'kernel': 'rbf'}

### Recriando o modelo com base nos melhores parâmetros encontrados no grid

In [35]:
# Dividindo o dataset em 2
kfold = KFold(n_splits=2, shuffle=True, random_state=0)

model = SVC(C=1000, gamma=0.1, kernel='rbf')

# Cross validation
results = cross_val_score(model, X_train, y_train, cv=kfold)
results.mean()

0.611375